<a href="https://colab.research.google.com/github/JuanRosales707/md-lab01-RC/blob/main/LAB_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_original = fetch_ucirepo(id=15)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_original.data.features
y = breast_cancer_wisconsin_original.data.targets

# metadata
print(breast_cancer_wisconsin_original.metadata)

# variable information
print(breast_cancer_wisconsin_original.variables)


{'uci_id': 15, 'name': 'Breast Cancer Wisconsin (Original)', 'repository_url': 'https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original', 'data_url': 'https://archive.ics.uci.edu/static/public/15/data.csv', 'abstract': 'Original Wisconsin Breast Cancer Database', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 699, 'num_features': 9, 'feature_types': ['Integer'], 'demographics': [], 'target_col': ['Class'], 'index_col': ['Sample_code_number'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1990, 'last_updated': 'Sun Mar 10 2024', 'dataset_doi': '10.24432/C5HP4Z', 'creators': ['WIlliam Wolberg'], 'intro_paper': None, 'additional_info': {'summary': "Samples arrive periodically as Dr. Wolberg reports his clinical cases. The database therefore reflects this chronological grouping of the data. This grouping information appears immediately below, having been removed fro

### **a. Calcule el information value (IV), tanto para el grupo de variable numéricas como categóricas y  excluya las que tenga un poder predictivo débil o menor. Además, separe la variable de  clasificación del resto de variables para luego obtener los datos de entrenamiento y prueba,  tomando de este último el 25% de datos.**

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Cargar dataset desde URL
link = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
columnas = ["ID", "Clump_Thickness", "Uniformity_Cell_Size", "Uniformity_Cell_Shape",
            "Marginal_Adhesion", "Single_Epithelial_Cell_Size", "Bare_Nuclei",
            "Bland_Chromatin", "Normal_Nucleoli", "Mitoses", "Class"]
data = pd.read_csv(link, names=columnas)

# Limpieza de datos
data = data.replace("?", np.nan)
data["Bare_Nuclei"] = pd.to_numeric(data["Bare_Nuclei"])
data = data.dropna().drop(columns=["ID"])
data["Class"] = data["Class"].map({2: 0, 4: 1})  # 0 = benigno, 1 = maligno

# Función para calcular Information Value (IV)
def obtener_iv(df, variable, objetivo, num_bins=5):
    temp = df[[variable, objetivo]].copy()
    if not pd.api.types.is_object_dtype(temp[variable]):
        temp[variable] = pd.qcut(temp[variable], q=num_bins, duplicates='drop')
    tabla = temp.groupby(variable)[objetivo].agg(['count', 'sum'])
    tabla.columns = ['Total', 'Positivos']
    tabla['Negativos'] = tabla['Total'] - tabla['Positivos']
    tabla['Perc_Pos'] = tabla['Positivos'] / tabla['Positivos'].sum()
    tabla['Perc_Neg'] = tabla['Negativos'] / tabla['Negativos'].sum()
    tabla['WoE'] = np.log(tabla['Perc_Pos'] / tabla['Perc_Neg']).replace([np.inf, -np.inf], 0)
    tabla['IV'] = (tabla['Perc_Pos'] - tabla['Perc_Neg']) * tabla['WoE']
    return tabla['IV'].sum()

# Calcular IV de todas las variables
iv_resultados = {}
for feature in data.columns.drop('Class'):
    iv_resultados[feature] = round(obtener_iv(data, feature, 'Class'), 4)

# Seleccionar variables con IV >= 0.1
iv_tabla = pd.DataFrame.from_dict(iv_resultados, orient='index', columns=['IV']).sort_values('IV', ascending=False)
iv_tabla_filtrada = iv_tabla.query('IV >= 0.1')
variables_finales = iv_tabla_filtrada.index.tolist()

# Definir X (predictoras) e y (objetivo)
X = data[variables_finales]
y = data["Class"]

# Dividir conjunto en entrenamiento (75%) y prueba (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Mostrar resumen
print("Variables seleccionadas:", variables_finales)
print(f"Entrenamiento: {X_train.shape} | Prueba: {X_test.shape}")

<ipython-input-18-7d5f8c3f6258>:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla = temp.groupby(variable)[objetivo].agg(['count', 'sum'])
<ipython-input-18-7d5f8c3f6258>:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla = temp.groupby(variable)[objetivo].agg(['count', 'sum'])
<ipython-input-18-7d5f8c3f6258>:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabla = temp.groupby(variable)[objetivo].agg(['count

Variables seleccionadas: ['Uniformity_Cell_Size', 'Bare_Nuclei', 'Uniformity_Cell_Shape', 'Bland_Chromatin', 'Single_Epithelial_Cell_Size', 'Clump_Thickness', 'Marginal_Adhesion', 'Normal_Nucleoli']
Entrenamiento: (512, 8) | Prueba: (171, 8)


### **b. Genere el modelo de regresión logística y evalúe la exclusión de variables mediante la  significancia de los coeficientes. Además, calcule las métricas de clasificación que se  implementaron en la parte práctica e interprete sus resultados más importantes.**

In [19]:
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# Agregar constante para incluir el intercepto en el modelo de statsmodels
X_train_constant = sm.add_constant(X_train)

# Ajustar el modelo de regresión logística con statsmodels
modelo_logit = sm.Logit(y_train, X_train_constant).fit()

# Mostrar resumen del modelo
print(modelo_logit.summary())

# Filtrar las variables significativas (p-value <= 0.05)
p_valores = modelo_logit.pvalues.drop("const")
variables_significativas = p_valores[p_valores <= 0.05].index.tolist()

# Crear nuevos conjuntos de entrenamiento y prueba con las variables significativas
X_train_significativas = X_train[variables_significativas]
X_test_significativas = X_test[variables_significativas]

# Entrenar el modelo usando scikit-learn
modelo_clf = LogisticRegression()
modelo_clf.fit(X_train_significativas, y_train)

# Predecir las clases y probabilidades
predicciones = modelo_clf.predict(X_test_significativas)
probabilidades = modelo_clf.predict_proba(X_test_significativas)[:, 1]

# Mostrar el reporte de clasificación
print("\n=== Reporte de Clasificación ===")
print(classification_report(y_test, predicciones))

# Imprimir la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix(y_test, predicciones))

# Calcular y mostrar AUC (Área bajo la curva ROC)
print(f"AUC: {roc_auc_score(y_test, probabilidades):.4f}")

Optimization terminated successfully.
         Current function value: 0.063072
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                  Class   No. Observations:                  512
Model:                          Logit   Df Residuals:                      503
Method:                           MLE   Df Model:                            8
Date:                Sat, 03 May 2025   Pseudo R-squ.:                  0.9025
Time:                        00:26:10   Log-Likelihood:                -32.293
converged:                       True   LL-Null:                       -331.37
Covariance Type:            nonrobust   LLR p-value:                5.843e-124
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                         -10.3115      1.440     -7.160      0.000  

### **c. Genere el modelo SVM, calcule sus métricas de clasificación y compárelas con las del modelo  de regresión logística para ver si hubo o no mejoras.**

In [39]:
# Paso 1: Importar librerías necesarias
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Paso 2: Dividir los datos (70% entrenamiento, 30% prueba)
X_train_sig, X_test_sig, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Paso 3: Estandarización
scaler = StandardScaler()
X_train_sig_scaled = scaler.fit_transform(X_train_sig)
X_test_sig_scaled = scaler.transform(X_test_sig)

# Paso 4: Crear y ajustar el modelo SVM (lineal con probabilidades)
svm_clf = SVC(kernel='linear', probability=True)
svm_clf.fit(X_train_sig_scaled, y_train)

# Paso 5: Predicciones
y_pred_svm = svm_clf.predict(X_test_sig_scaled)
y_prob_svm = svm_clf.predict_proba(X_test_sig_scaled)[:, 1]

# Paso 6: Métricas
print("\n=== Reporte de Clasificación SVM ===")
print(classification_report(y_test, y_pred_svm))
print("Matriz de Confusión SVM:")
print(confusion_matrix(y_test, y_pred_svm))
print(f"AUC (SVM): {roc_auc_score(y_test, y_prob_svm):.4f}")



=== Reporte de Clasificación SVM ===
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       127
           1       0.97      0.94      0.95        78

    accuracy                           0.97       205
   macro avg       0.97      0.96      0.96       205
weighted avg       0.97      0.97      0.97       205

Matriz de Confusión SVM:
[[125   2]
 [  5  73]]
AUC (SVM): 0.9963
